In [ ]:
from ase.io import read as ase_read
import numpy as np
#from vesin import ase_neighbor_list
import json
from collections import Counter
import copy
from ase.io import write as ase_write

In [ ]:
input_xyz = "jp_dio-orig_min4.xyz"
min_orig_atoms = ase_read(input_xyz)
min_orig_out = copy.deepcopy(min_orig_atoms)

NOTE: The neighbor list I'm going to use is from the non-minimized orig_dio structure. However, I'm working on the minimized version. Thus there will be some discrepency between the neighborlist, but it shouldn't be large. Just track the max distance of the atoms included in the neighbors

In [ ]:
# Setting a high buffer for the vesin cutoff. Or maybe I just won't do this
buffer = 5.0
rcut = 10.0 + buffer

In [ ]:
o_idxs = [idx for idx in range(len(min_orig_atoms)) if min_orig_atoms[idx].symbol == "O"]

In [ ]:
min_orig_atoms[0].symbol

In [ ]:
min_orig_atoms[o_idxs]

In [ ]:
nnlist_data = np.load("orig_dio_polycrystal_neighborlist10dot4.npz")
nn_i, nn_j, nn_S = nnlist_data["i"] , nnlist_data["j"], nnlist_data["S"]
nn_dict = {}
for k in range(len(nn_i)):
    iidx = int(nn_i[k])
    if iidx not in nn_dict:
        nn_dict[iidx] = []
    nn_dict[iidx].append(int(nn_j[k]))

In [ ]:
len(nn_dict)
#len(min_orig_atoms)

In [ ]:
nn_dict

In [ ]:
with open("noOidx2orig.json", "r") as f:
    index_map = json.load(f)

# I want to reverse this, i.e. go from orig to noO
index_map = {int(v): int(k) for k,v in index_map.items()}

In [ ]:
index_map

In [ ]:
grain_ptm_data = np.load("grains_ptm_111025_min4.npz")
noO_grains = grain_ptm_data["grains"]
noO_ptm_types = grain_ptm_data["ptm_types"]

In [ ]:
len(noO_grains)

In [ ]:
max([v for k,v in index_map.items()])

In [ ]:
oatom_envs = []
for idx in o_idxs[:10]:
    print(f"Iteration {idx}")
    neighbor_idxs = nn_dict[idx]
    #print(neighbor_idxs)

    local_grains = []
    local_ptm_types = []
    o_count = 0
    for nidx in neighbor_idxs:
        atm = min_orig_atoms[nidx]
        if atm.symbol == "O":
            o_count += 1
            continue
        local_grains.append(int(noO_grains[index_map[nidx]]))
        local_ptm_types.append(int(noO_ptm_types[index_map[nidx]]))
    num_Hf_neighs = len(local_grains)

    num_hcp = local_ptm_types.count(2)
    num_other = local_ptm_types.count(0)
    fract_hcp = num_hcp/num_Hf_neighs
    fract_hcp_c = 1 -fract_hcp
    fract_other = num_other/num_Hf_neighs

    grain_counter = dict(Counter(local_grains))
    grain_fract = {k: v/num_Hf_neighs for k,v in grain_counter.items()}

    #print(o_count)
    #print(local_grains)
    #print(local_ptm_types)
    #print(f"fract hcp: {fract_hcp}")
    #print(f"fract hcp complement: {fract_hcp_c}")
    #print(f"fract other: {fract_other}")
    #print(grain_counter)
    #print(grain_fract)

    oatom_envs.append({"index" : idx,
                       "o_count" : o_count,
                       "fract_hcp" : fract_hcp,
                       "fract_hcp_c" : fract_hcp_c,
                       "fract_other": fract_other,
                       "grain_fract": grain_fract})

    #distance calculation
    #i,j,S = ase_neighbor_list("ijS", orig_polycrystal, cutoff=10.4)
    #nn_cluster_pos = min_orig_atoms[neighbor_idxs].positions
    #nn_cluster_


In [ ]:
for neigh_summary in oatom_envs:
    print(neigh_summary["grain_fract"])

In [ ]:
oatom_envs[-2]

In [ ]:
isneighbor = np.zeros(len(min_orig_atoms))
isneighbor[nn_dict[90839]] = 1
isneighbor[90839] = 1

In [ ]:
min_orig_out.set_array("is_neighbor", isneighbor)

In [ ]:
ref_symbols = min_orig_out.get_chemical_symbols().copy()
new_symbols = ref_symbols.copy()
new_symbols[90839] = "Np"
min_orig_out.set_chemical_symbols(new_symbols)

In [ ]:
ase_write("temp_data2.xyz", min_orig_out, format="extxyz")

In [ ]:
xyz_grain_idxs = []
xyz_ptm_types = []

for i,atm in enumerate(min_orig_atoms):
    if atm.symbol == "O":
        xyz_grain_idxs.append(-1)
        xyz_ptm_types.append(-1)
        continue

    xyz_grain_idxs.append(noO_grains[index_map[i]])
    xyz_ptm_types.append(noO_ptm_types[index_map[i]])


In [ ]:
np.array(xyz_grain_idxs)

In [ ]:
min_orig_out.set_array("grain_index", np.array(xyz_grain_idxs))
min_orig_out.set_array("ptm_type", np.array(xyz_ptm_types))

In [ ]:
ase_write("temp_data3.xyz", min_orig_out, format="extxyz")

In [ ]:
max(noO_grains)